In [323]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
%matplotlib inline
import yfinance
from utility import gain, plot_model_on_data, roi, prepare_data, print_eval

In [324]:
nasdaq = yfinance.Ticker("^IXIC")

In [325]:
data = nasdaq.history(start="2011-01-01", end="2021-12-31", actions=False)

In [326]:
open = data["Open"]
close = data["Close"]

features= {
    "Open": open
}

X = pd.DataFrame(features)
y = close

def split_data(X, y, split):
    lowerRange = X.index.year < split
    upperRange = (X.index.year > split-1) & (X.index.year < split+1)
    X_train = X.loc[lowerRange]
    y_train = y.loc[lowerRange]
    X_val = X.loc[upperRange]
    y_val = y.loc[upperRange]
    return X_train, X_val, y_train, y_val

X_train, X_val, y_train, y_val = split_data(X, y, 2012)

In [327]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train,y_train)
model.score(X_val, y_val)

0.9522082278765175

In [328]:
delta = close - open

In [329]:

features = {
    "Open": open,
    "OpenShift1": open.shift(1),
    "OpenShift2": open.shift(2),
    "OpenShift3": open.shift(3),
    "OpenShift4": open.shift(4),
}

In [330]:
X, y = prepare_data(features, delta)
X_train, X_val, y_train, y_val = split_data(X, y, 2021)
model = LinearRegression()
model.fit(X_train, y_train)
print_eval(X_val, y_val, model, open)

Gain: 58.50€
 ROI: 0.407%


## Aggiungiamo Features

In [331]:
def calculate_ema(prices, days, smoothing=2):
    ema = [sum(prices[:days]) / days]
    for price in prices[days:]:
        ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
    return ema

def computeMACD (df, n_fast, n_slow, n_smooth):
    data = df['Open']
    
    fastEMA = data.ewm(span=n_fast, min_periods=n_slow).mean()
    slowEMA = data.ewm(span=n_slow, min_periods=n_slow).mean()
    MACD = pd.Series(fastEMA-slowEMA, name = 'MACD')
    df = df.join(MACD)
    
    return df


In [332]:
def addEMA(first, second):
    ema1 = calculate_ema(data['Open'], first)
    ema2 = calculate_ema(data['Open'], second)

    ema = (np.array(ema2) - np.array(ema1[(second-first):])) / np.array(ema2)
    ema = pd.Series(data=ema, index=data[(second-1):].index)
    data[f"EMA{first}-{second}"] = ema
    features[f"EMA{first}-{second}"] = data[f"EMA{first}-{second}"]
    

addEMA(5,20)
addEMA(8,15)
addEMA(20,50)

data = computeMACD(data, 12, 26, 9)

features["MACD"] = data["MACD"]

In [338]:
X, y = prepare_data(features, delta)
X_train, X_val, y_train, y_val = split_data(X, y, 2015)
model = LinearRegression()
model.fit(X_train, y_train)
print_eval(X_val, y_val, model, open)

Gain: -96.25€
 ROI: -1.946%


In [334]:
features

{'Open': Date
 2010-12-31     2660.580078
 2011-01-03     2676.649902
 2011-01-04     2699.860107
 2011-01-05     2673.909912
 2011-01-06     2704.379883
                   ...     
 2021-12-23    15544.790039
 2021-12-27    15696.830078
 2021-12-28    15895.200195
 2021-12-29    15794.919922
 2021-12-30    15758.980469
 Name: Open, Length: 2769, dtype: float64,
 'OpenShift1': Date
 2010-12-31             NaN
 2011-01-03     2660.580078
 2011-01-04     2676.649902
 2011-01-05     2699.860107
 2011-01-06     2673.909912
                   ...     
 2021-12-23    15319.160156
 2021-12-27    15544.790039
 2021-12-28    15696.830078
 2021-12-29    15895.200195
 2021-12-30    15794.919922
 Name: Open, Length: 2769, dtype: float64,
 'OpenShift2': Date
 2010-12-31             NaN
 2011-01-03             NaN
 2011-01-04     2660.580078
 2011-01-05     2676.649902
 2011-01-06     2699.860107
                   ...     
 2021-12-23    15140.429688
 2021-12-27    15319.160156
 2021-12-28    15544